In [1]:
from rayllm_batch.workload import ChatWorkloadBase
from typing import Optional, Dict, Any
import ray 
from ray.data.dataset import Dataset
from dataclasses import dataclass, field


@dataclass
class CNNDailySummary(ChatWorkloadBase):
    """The CNN/DailyMail summarization workload."""

    # We directly load the dataset from Hugging Face.
    dataset_file: Optional[str] = None
    # We will load only a portion of the dataset to run inference faster for the tutorial.
    dataset_fraction: float = 0.0005 # 0.2% of the 300K entries.
    # The sampling params for the LLM inference workload.
    sampling_params: Dict[str, Any] = field(default_factory=lambda: {"max_tokens": 200})

    def load_dataset(self) -> Dataset:
        # Load the dataset from Hugging Face into Ray Data.
        import datasets  # type: ignore

        df = datasets.load_dataset("cnn_dailymail", "3.0.0")
        return ray.data.from_huggingface(df["train"])

    def parse_row(self, row: dict[str, Any]) -> dict[str, Any]:
        # Parse the row into the format expected by the model.
        # We will use the article as the user prompt, and ask the model to 
        # generate a summary with the system prompt.
        return {
            "messages": [
                {
                    "role": "system",
                    "content": "You are a commentator. Your task is to "
                    "summarize highlights from article.",
                },
                {
                    "role": "user",
                    "content": f"# Article:\n{row['article']}\n\n"
                    "#Instructions:\nIn clear and concise language, "
                    "summarize the highlights presented in the article.",
                },
            ]
        }

In [2]:
# Prompts the user for Hugging Face token if required by the model.
from util.utils import prompt_for_hugging_face_token
HF_TOKEN = prompt_for_hugging_face_token("meta-llama/Meta-Llama-3.1-8B-Instruct")

Successfully read cached token at huggingface_token.txt.


In [3]:
from rayllm_batch import init_engine_from_config
# Read the model configs from the path.
model_config_path = "configs/llama-3.1-8b-a10g.yaml"

# One could potentially override the engine configs by passing in a dictionary here.
override = {"runtime_env": {"env_vars": {"HF_TOKEN": HF_TOKEN}}} # Override Ray's runtime env to include the Hugging Face token. Ray is being used under the hood to orchestrate the inference pipeline.
engine_config = init_engine_from_config(config=model_config_path, override=override)


In [4]:
from rayllm_batch import RayLLMBatch


workload = CNNDailySummary()
batch = RayLLMBatch(
    engine_cfg=engine_config,
    workload=workload,
    # Specify the batch size for inference. Set the batch size to as large as possible without running out of memory.
    # If you encounter out-of-memory errors, decreasing batch_size may help. 
    batch_size=None,
    # Set the number of replicas to use for the inference. Each replica will run one instance of inference pipeline.
    num_replicas=1,
)


# This will runs until completion.
ds = batch.run()


# Read the results
gen_texts = [r["generated_text"] for r in ds.take_all()]
print(gen_texts)

2024-09-24 00:44:00,222	INFO worker.py:1601 -- Connecting to existing Ray cluster at address: 10.0.27.195:6379...
2024-09-24 00:44:00,230	INFO worker.py:1777 -- Connected to Ray cluster. View the dashboard at https://session-5abw5nxh2chqdlhxiklnw6cu1k.i.anyscaleuserdata.com 
2024-09-24 00:44:00,233	INFO packaging.py:359 -- Pushing file package 'gcs://_ray_pkg_0fa039546672a8668c01da562dca2994d8a900a9.zip' (0.65MiB) to Ray cluster...
2024-09-24 00:44:00,240	INFO packaging.py:372 -- Successfully pushed file package 'gcs://_ray_pkg_0fa039546672a8668c01da562dca2994d8a900a9.zip'.


Parquet Files Sample 0:   0%|          | 0.00/2.00 [00:00<?, ? file/s]

2024-09-24 00:44:05,980	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-09-24_00-28-42_051202_2516/logs/ray-data
2024-09-24 00:44:05,981	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> TaskPoolMapOperator[MapBatches(random_sample)->Map(CNNDailySummary.parse_row)] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- ReadParquet->SplitBlocks(11) 1: 0.00 row [00:00, ? row/s]

...

Total tokens processed: 134842
Engine throughput (tokens/s): 1993.59
Projected 1M token time (mins): 9.08
2024-09-24 00:45:51,517	INFO streaming_executor.py:108 -- Starting execution of Dataset. Full logs are in /tmp/ray/session_2024-09-24_00-28-42_051202_2516/logs/ray-data
2024-09-24 00:45:51,518	INFO streaming_executor.py:109 -- Execution plan of Dataset: InputDataBuffer[Input] -> LimitOperator[limit=1]


Running 0: 0.00 row [00:00, ? row/s]

- limit=1 1: 0.00 row [00:00, ? row/s]

"In Kabul, Afghanistan, over 1,000 bottles of alcohol were destroyed as part of a crackdown on smuggling and sales. The bottles were confiscated from Afghan sources over a two-year period, mostly from international hotels and handed over to customs officials who burnt them. This movemade facilitates enforcement of Afghanistan's ban on alcohol due to its largely Muslim population, with only a few specifically designated areas allowing its sale for foreigners."

(autoscaler +4m31s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.


In [7]:
gen_texts = [r["generated_text"] for r in ds.take_all()]

In [8]:
gen_texts

["In Kabul, Afghanistan, over 1,000 bottles of alcohol were destroyed as part of a crackdown on smuggling and sales. The bottles were confiscated from Afghan sources over a two-year period, mostly from international hotels and handed over to customs officials who burnt them. This movemade facilitates enforcement of Afghanistan's ban on alcohol due to its largely Muslim population, with only a few specifically designated areas allowing its sale for foreigners.",
 'Here are the article\'s highlights summarized:\n\nPhotographer Eiko Jones was taking underwater pictures of plants near Campbell River in Canada when he was suddenly surrounded by thousands of tadpoles swimming at the Cedar Lake. The 41-year-old photographer was initially startled by the large group of tadpoles, which appeared as a "cloud" swimming around him for about ten minutes. The rare sighting was a unique experience for Jones, who stated that he had never seen or heard of tadpoles schooling before.',
 'Two high-ranking 